Mở lại dữ liệu đã đọc

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
import pickle
X_data = pickle.load(open('data/X_data.pkl', 'rb'))
y_data = pickle.load(open('data/y_data.pkl', 'rb'))

X_test = pickle.load(open('data/X_test.pkl', 'rb'))
y_test = pickle.load(open('data/y_test.pkl', 'rb'))

In [4]:
X_data[0]

'thành_lập dự_án policy phòng_chống hiv aids vn nlđ quỹ hỗ_trợ khẩn_cấp về aids của hoa kỳ vừa thành_lập dự_án policy tại vn với cam_kết hỗ_trợ chính_phủ và nhân_dân vn đối_phó_hiv aids dự_án có nhiệm_vụ chính là cải_thiện công_tác phòng_chống hiv aids thông_qua các lĩnh_vực xây_dựng chính_sách rà_soát các văn_bản pháp_luật xây_dựng chiến_lược quảng_bá xây_dựng chương_trình đào_tạo về phòng_chống hiv aids lên kế_hoạch bố_trí nguồn_lực huấn_luyện và nghiên_cứu về phương_tiện truyền_thông đại_chúng tổ_chức các hoạt_động nhằm giảm kỳ_thị và phân_biệt đối_xử đối_với người có hiv aids theo ttxvn dự_án policy đặc_biệt quan_tâm đến công_tác truyền_thông phòng_chống hiv aids coi đây là một biện_pháp tích_cực và hữu_hiệu trong việc phòng_chống có hiệu_quả hiv aids thời_gian tới dự_án policy sẽ tiếp_tục tổ_chức các hoạt_động nhằm nâng cao nhận_thức cho những người có trách_nhiệm với công_tác chỉ_đạo phòng_chống hiv aids'

## Bag - of - Words
- `\w`: Returns a match where the string contains any word characters (characters from a to Z, digits from 0-9, and the underscore _ character)
- The `r` means that the string is to be treated as a raw string, which means all escape codes will be ignored.
- `r'\w{1,}'`: từ chứa một ký tự chữ được lặp lại 1 lần hoặc hơn

In [12]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)

## TF-IDF Vectors
Trong các văn bản tương tự về chủ đề, lĩnh vực, các từ vựng thông dụng (generic term) sẽ xuất hiện nhiều, có thể lấn át đi các từ quan trọng, nếu chỉ đơn giản represent bằng word count.
### Ý tưởng:
- Các từ nào xuất hiện càng nhiều trong $n$ văn bản, khả năng nó là generic term càng cao, nên mình sẽ làm giảm bớt "mức độ ảnh hưởng" của nó đi bằng cho nó điểm thấp hơn, còn từ nào xuất hiện ở càng ít văn bản thì có khả năng nó là từ key để phân loại, ta cho điểm cao lên.
    - Công thức sử dụng công thức $IDF$
- Tuy nhiên, trong các văn bản một từ `t` có xuất hiện, số lượng từ vựng ở mỗi văn bản đó có thể chênh lệch nhau nhiều (một văn bản 5000 chữ so với một văn bản 10 chữ), thì nó cũng chưa chắc là từ key (mức độ quan trọng của cùng một từ khác nhau ở các văn bản khác nhau). Vì vậy ta cần sử dụng thêm công thức $TF$ để tính tần suất xuất hiện của từ ý trong một văn bản cụ thể $A$

$ TF(t)$ = (Number of times term t appears in a document) / (Total number of terms in the document)

$IDF(t)$ = log_e(Total number of documents / Number of documents with term t in it)

- TF-IDF Vectors can be generated at different levels of input tokens (words, characters, n-grams)

If a word appears in all the documents, we want it at the bottom of the range of 0–1. So, a logarithmic scale intuitively makes sense to be used here as log 1 is 0. However, there are some practical considerations such as avoiding the infamous divide by 0 error, 1 is added to the denominator.

Inverse Document frequency for the default settings in TF IDF vectorizer in sklearn is calculated as below (default settings have smooth_idf=True that adds “1” to the numerator and denominator as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions).

a. Word Level TF-IDF : Thực hiện tính toán dựa trên mỗi thành phần là một từ riêng lẻ

In [13]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)

b. N-gram Level TF-IDF : Kết hợp n thành phần (từ) liên tiếp nhau

In [14]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

## Sử dụng thuật toán SVD (singular value decomposition) nhằm mục đích giảm chiều dữ liệu của ma trận
a. Word Level TF-IDF : Thực hiện tính toán dựa trên mỗi thành phần là một từ riêng lẻ

In [15]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)


X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

b. N-gram Level TF-IDF : Kết hợp n thành phần (từ) liên tiếp nhau

In [16]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)

# Label Encoder

In [17]:
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_ # kết quả: array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
                 #                 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
                 #                 'Vi tinh'], dtype='<U16')


array(['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh',
       'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa',
       'Vi tinh'], dtype='<U16')

# Model

In [18]:
from sklearn.model_selection import train_test_split
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))